### Imports and configuration

In [4]:
import os
from pathlib import Path

import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Adjust year/month
YEAR = 2024
MONTH = 1

# Raw data stored locally
DATA_DIR = Path("..") / "data"

# Taxi Trips data path
TAXI_FILE_NAME = f"yellow_tripdata_{YEAR}-{MONTH:02d}.parquet"
TAXI_FILE_PATH = DATA_DIR / TAXI_FILE_NAME

# Reference data path
REF_FILE_NAME = "taxi_zone_lookup.csv"
REF_FILE_PATH = DATA_DIR / REF_FILE_NAME

print(TAXI_FILE_PATH)
print(REF_FILE_PATH)

../data/yellow_tripdata_2024-01.parquet
../data/taxi_zone_lookup.csv


### Load Zone reference Data

In [5]:
df_ref = pd.read_csv(REF_FILE_PATH)
df_ref.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [6]:
df_ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


### Load Taxi Trips Data

In [7]:
df_taxi = pd.read_parquet(TAXI_FILE_PATH)
df_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


### Basic structure

In [8]:
df_taxi.shape  # (rows, columns)

(2964624, 19)

In [5]:
df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

### Preview schemas and unique values for key categoricals

In [9]:
# Overview of column names
df_taxi.columns.tolist()

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [8]:
# VendorID, RateCodeID, Payment_type, Store_and_fwd_flag etc.
categorical_cols = [
    "VendorID",
    "RatecodeID",       # or "RateCodeID" depending on schema
    "payment_type",
    "store_and_fwd_flag",
]

for col in categorical_cols:
    if col in df_taxi.columns:
        display(f"Value counts for {col}")
        display(df_taxi[col].value_counts(dropna=False).head(20))

'Value counts for VendorID'

VendorID
2    2234632
1     729732
6        260
Name: count, dtype: int64

'Value counts for RatecodeID'

RatecodeID
1.0     2663350
NaN      140162
2.0       98713
99.0      28663
5.0       19410
3.0        7954
4.0        6365
6.0           7
Name: count, dtype: int64

'Value counts for payment_type'

payment_type
1    2319046
2     439191
0     140162
4      46628
3      19597
Name: count, dtype: int64

'Value counts for store_and_fwd_flag'

store_and_fwd_flag
N       2813126
None     140162
Y         11336
Name: count, dtype: int64

### Basic descriptive statistics

In [10]:
# Numeric stats only
df_taxi.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,2.964624e+06,2964624,2964624,2.824462e+06,2.964624e+06,2.824462e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.824462e+06,2.824462e+06
mean,1.754204e+00,2024-01-17 00:46:36.431092,2024-01-17 01:02:13.208130,1.339281e+00,3.652169e+00,2.069359e+00,1.660179e+02,1.651167e+02,1.161271e+00,1.817506e+01,1.451598e+00,4.833823e-01,3.335870e+00,5.270212e-01,9.756319e-01,2.680150e+01,2.256122e+00,1.411611e-01
min,1.000000e+00,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-8.990000e+02,-7.500000e+00,-5.000000e-01,-8.000000e+01,-8.000000e+01,-1.000000e+00,-9.000000e+02,-2.500000e+00,-1.750000e+00
25%,2.000000e+00,2024-01-09 15:59:19.750000,2024-01-09 16:16:23,1.000000e+00,1.000000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.538000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2024-01-17 10:45:37.500000,2024-01-17 11:03:51.500000,1.000000e+00,1.680000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.700000e+00,0.000000e+00,1.000000e+00,2.010000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2024-01-24 18:23:52.250000,2024-01-24 18:40:29,1.000000e+00,3.110000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.120000e+00,0.000000e+00,1.000000e+00,2.856000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,2024-02-01 00:01:15,2024-02-02 13:56:52,9.000000e+00,3.127223e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,5.000000e+03,1.425000e+01,4.000000e+00,4.280000e+02,1.159200e+02,1.000000e+00,5.000000e+03,2.500000e+00,1.750000e+00
std,4.325902e-01,NaN,NaN,8.502817e-01,2.254626e+02,9.823219e+00,6.362391e+01,6.931535e+01,5.808686e-01,1.894955e+01,1.804102e+00,1.177600e-01,3.896551e+00,2.128310e+00,2.183645e-01,2.338558e+01,8.232747e-01,4.876239e-01


In [11]:
# Include all (will show counts, unique, top, freq for object columns)
df_taxi.describe(include="all")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,2.964624e+06,2964624,2964624,2.824462e+06,2.964624e+06,2.824462e+06,2824462,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.964624e+06,2.824462e+06,2.824462e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,2813126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.754204e+00,2024-01-17 00:46:36.431092,2024-01-17 01:02:13.208130,1.339281e+00,3.652169e+00,2.069359e+00,NaN,1.660179e+02,1.651167e+02,1.161271e+00,1.817506e+01,1.451598e+00,4.833823e-01,3.335870e+00,5.270212e-01,9.756319e-01,2.680150e+01,2.256122e+00,1.411611e-01
min,1.000000e+00,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000000e+00,0.000000e+00,1.000000e+00,NaN,1.000000e+00,1.000000e+00,0.000000e+00,-8.990000e+02,-7.500000e+00,-5.000000e-01,-8.000000e+01,-8.000000e+01,-1.000000e+00,-9.000000e+02,-2.500000e+00,-1.750000e+00
25%,2.000000e+00,2024-01-09 15:59:19.750000,2024-01-09 16:16:23,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.538000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2024-01-17 10:45:37.500000,2024-01-17 11:03:51.500000,1.000000e+00,1.680000e+00,1.000000e+00,NaN,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.700000e+00,0.000000e+00,1.000000e+00,2.010000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2024-01-24 18:23:52.250000,2024-01-24 18:40:29,1.000000e+00,3.110000e+00,1.000000e+00,NaN,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.120000e+00,0.000000e+00,1.000000e+00,2.856000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,2024-02-01 00:01:15,2024-02-02 13:56:52,9.000000e+00,3.127223e+05,9.900000e+01,NaN,2.650000e+02,2.650000e+02,4.000000e+00,5.000000e+03,1.425000e+01,4.000000e+00,4.280000e+02,1.159200e+02,1.000000e+00,5.000000e+03,2.500000e+00,1.750000e+00


### Missing values overview

In [12]:
null_counts = df_taxi.isna().sum().sort_values(ascending=False)
null_ratio = (df_taxi.isna().mean() * 100).sort_values(ascending=False)

missing_summary = pd.concat(
    [null_counts.rename("null_count"), null_ratio.rename("null_pct")],
    axis=1,
)

missing_summary


,null_count,null_pct
Airport_fee,140162,4.727817
congestion_surcharge,140162,4.727817
passenger_count,140162,4.727817
RatecodeID,140162,4.727817
store_and_fwd_flag,140162,4.727817
extra,0,0.000000
total_amount,0,0.000000
improvement_surcharge,0,0.000000
tolls_amount,0,0.000000
tip_amount,0,0.000000


### Basic quality checks

In [13]:
# Non-positive trip distances
if "trip_distance" in df_taxi.columns:
    df_taxi[df_taxi["trip_distance"] <= 0].head()

In [14]:
# Negative or zero fares / total_amount
problem_fares = []
for col in ["fare_amount", "total_amount"]:
    if col in df_taxi.columns:
        problem_fares.append(
            df_taxi[df_taxi[col] <= 0][[col]].assign(problem_col=col)
        )

if problem_fares:
    pd.concat(problem_fares, axis=0).head()

In [15]:
# Duplicated rows check (fully duplicated)
df_taxi.duplicated().sum()

np.int64(0)

In [16]:
# Example of potential business key duplicates if present
potential_key = [
    c for c in [
        "VendorID",
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
    ]
    if c in df_taxi.columns
]

if potential_key:
    df_taxi.duplicated(subset=potential_key).sum()

### Datetime features

In [17]:
# Ensure datetimes are proper dtype
for col in ["tpep_pickup_datetime", "tpep_dropoff_datetime"]:
    if col in df_taxi.columns and not pd.api.types.is_datetime64_any_dtype(df_taxi[col]):
        df_taxi[col] = pd.to_datetime(df_taxi[col])

[df_taxi[col].dt.date.min() for col in df_taxi.columns if col.endswith("_datetime") and pd.api.types.is_datetime64_any_dtype(df_taxi[col])]

[datetime.date(2002, 12, 31), datetime.date(2002, 12, 31)]

In [18]:
# Trip duration in minutes
df_taxi["trip_duration_min"] = (
        (df_taxi["tpep_dropoff_datetime"] - df_taxi["tpep_pickup_datetime"])
        .dt.total_seconds() / 60.0
    )
df_taxi["trip_duration_min"].describe()

count    2.964624e+06
mean     1.561295e+01
std      3.485105e+01
min     -1.356667e+01
25%      7.150000e+00
50%      1.163333e+01
75%      1.868333e+01
max      9.455400e+03
Name: trip_duration_min, dtype: float64

### Distributions

In [30]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_white"

In [31]:
# Total amount distribution
if "total_amount" in df_taxi.columns:
    s = df_taxi["total_amount"].dropna().to_numpy()

    counts, edges = np.histogram(s, bins=100)
    centers = (edges[:-1] + edges[1:]) / 2

    fig = go.Figure(go.Bar(x=centers, y=counts))
    fig.update_layout(title="Total amount distribution", xaxis_title="total_amount", yaxis_title="count")
    fig.update_yaxes(type="log")
    fig.show()

In [32]:
# Tip amount distribution
if "tip_amount" in df_taxi.columns:
    s = df_taxi["tip_amount"].dropna().to_numpy()
    counts, edges = np.histogram(s, bins=100)
    centers = (edges[:-1] + edges[1:]) / 2

    fig = go.Figure(go.Bar(x=centers, y=counts))
    fig.update_layout(title="Tip amount distribution", xaxis_title="tip_amount", yaxis_title="count")
    fig.update_yaxes(type="log")
    fig.show()

In [33]:
# Passenger count frequencies
if "passenger_count" in df_taxi.columns:
    vc = df_taxi["passenger_count"].value_counts(dropna=False).sort_index()
    fig = go.Figure(go.Bar(x=vc.index.astype(str), y=vc.values))
    fig.update_layout(title="Passenger count distribution", xaxis_title="passenger_count", yaxis_title="trips")
    fig.show()

In [34]:
# Payment type breakdown
payment_type_dim = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip",
}

if "payment_type" in df_taxi.columns:
    vc = df_taxi["payment_type"].value_counts(dropna=False).sort_index()

    labels = []
    for code in vc.index:
        if pd.isna(code):
            labels.append("NaN (missing)")
        else:
            code_int = int(code)
            labels.append(f"{code_int} - {payment_type_dim.get(code_int, 'Other/Unknown')}")

    fig = go.Figure(go.Bar(x=labels, y=vc.values))
    fig.update_layout(title="Payment type distribution", xaxis_title="payment_type", yaxis_title="trips")
    fig.show()

In [37]:
# Rate code distribution
ratecode_dim = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau/Westchester",
    5: "Negotiated fare",
    6: "Group ride",
    99: "Unknown/Null",
}

rate_col = next((c for c in ["RatecodeID", "RateCodeID"] if c in df_taxi.columns), None)
if rate_col is not None:
    vc = df_taxi[rate_col].value_counts(dropna=False).sort_index()

    labels = []
    for code in vc.index:
        if pd.isna(code):
            labels.append("NaN")
        else:
            code_int = int(code)
            labels.append(f"{code_int} - {ratecode_dim.get(code_int, 'Other/Unknown')}")

    fig = go.Figure(go.Bar(x=labels, y=vc.values))
    fig.update_layout(title=f"{rate_col} distribution", xaxis_title=rate_col, yaxis_title="trips")
    fig.show()

### Temporal patterns

In [38]:
# Trips per hour of day
if "tpep_pickup_datetime" in df_taxi.columns:
    pickup_hour = df_taxi["tpep_pickup_datetime"].dt.hour
    vc = pickup_hour.value_counts(dropna=False).sort_index()

    x = vc.index.to_numpy()
    y = vc.values

    fig = go.Figure(go.Bar(x=x, y=y))
    fig.update_layout(title="Trips per pickup hour of day", xaxis_title="pickup_hour", yaxis_title="trips")
    fig.update_xaxes(dtick=1)
    fig.show()

In [44]:
# Average total amount by pickup hour
if {"tpep_pickup_datetime", "total_amount"}.issubset(df_taxi.columns):
    hour = df_taxi["tpep_pickup_datetime"].dt.hour
    g = df_taxi.groupby(hour)["total_amount"].mean().reindex(range(24))

    fig = go.Figure(go.Scatter(x=g.index.to_numpy(), y=g.values, mode="lines+markers"))
    fig.update_layout(title="Average total amount by pickup hour", xaxis_title="pickup hour", yaxis_title="avg total_amount")
    fig.update_xaxes(dtick=1)
    fig.show()


### Reference Zone

In [48]:
# Top pickup / dropoff zones (join via df_ref)
loc_col = next((c for c in ["LocationID", "location_id", "locationid"] if c in df_ref.columns), None)
if loc_col is None:
    raise KeyError("df_ref must contain a LocationID column (e.g., 'LocationID').")

zone_col = next((c for c in ["Zone", "zone"] if c in df_ref.columns), None)
borough_col = next((c for c in ["Borough", "borough"] if c in df_ref.columns), None)

ref = df_ref.copy()
ref[loc_col] = pd.to_numeric(ref[loc_col], errors="coerce").astype("Int64")

if zone_col is None:
    ref["zone_label"] = ref[loc_col].astype(str)
else:
    if borough_col is None:
        ref["zone_label"] = ref[zone_col].astype(str)
    else:
        ref["zone_label"] = ref[borough_col].astype(str) + " — " + ref[zone_col].astype(str)

In [49]:
# Pickups
if "PULocationID" in df_taxi.columns:
    pu = df_taxi["PULocationID"].value_counts().rename_axis("LocationID").reset_index(name="trips")
    pu["LocationID"] = pu["LocationID"].astype("Int64")
    pu = pu.merge(ref[[loc_col, "zone_label"]].rename(columns={loc_col: "LocationID"}), on="LocationID", how="left")
    pu["zone_label"] = pu["zone_label"].fillna(pu["LocationID"].astype(str))

    top_pu = pu.head(20).iloc[::-1]
    fig = go.Figure(go.Bar(x=top_pu["trips"], y=top_pu["zone_label"], orientation="h"))
    fig.update_layout(title="Top 20 pickup zones", xaxis_title="trips", yaxis_title="")
    fig.show()

In [50]:
# Dropoffs
if "DOLocationID" in df_taxi.columns:
    do = df_taxi["DOLocationID"].value_counts().rename_axis("LocationID").reset_index(name="trips")
    do["LocationID"] = do["LocationID"].astype("Int64")
    do = do.merge(ref[[loc_col, "zone_label"]].rename(columns={loc_col: "LocationID"}), on="LocationID", how="left")
    do["zone_label"] = do["zone_label"].fillna(do["LocationID"].astype(str))

    top_do = do.head(20).iloc[::-1]
    fig = go.Figure(go.Bar(x=top_do["trips"], y=top_do["zone_label"], orientation="h"))
    fig.update_layout(title="Top 20 dropoff zones", xaxis_title="trips", yaxis_title="")
    fig.show()